## Analysis With Folium

**Objectives**


* Mark all launch sites on a map
* Mark the success/failed launches for each site on the map
* Calculate the distances between a launch site to its proximities

In [ ]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [ ]:
# import libraries
import folium
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

Mark All Launch Sites on Map

In [ ]:
# spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [ ]:
# take a look at what are the coordinates for each site.
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
# Create and add folium.Circle and folium.Marker for each launch site on the site map
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Iterate over each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    site_map.add_child(circle)
    
    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    site_map.add_child(marker)

site_map

Mark the success/failed launches for each site on the map


In [ ]:
# create markers for all launch records. 
#If a launch was successful (class=1) and use a green marker
#if a launch was failed, we use a red marker (class=0)
# create a marker cluster object
marker_cluster = MarkerCluster()

In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)

In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='darkblue', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Calculate the distances between a launch site to its proximities


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678

# Closest coastline point
coastline_lat = 28.56342
coastline_lon = -80.56744
# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# starting point of the map
m = folium.Map(location = [launch_site_lat, launch_site_lon], zoom_start = 15)

# create a marker for the launch site
folium.Marker(location = [launch_site_lat, launch_site_lon],
              icon=folium.Icon(color='red', icon='rocket', prefix='fa')
             ).add_to(m)                    

# create a marker for the coastline point
distance_marker = folium.Marker(location = [launch_site_lat, launch_site_lon],
                                icon = DivIcon(
                                    icon_size = (20, 20),
                                    icon_anchor = (0, 0),
                                    html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
                                              )
                                 )
distance_marker.add_to(m) 

# display the map
m

In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

import folium

# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678
coastline_lat = 28.56342
coastline_lon = -80.56744

# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)


marker_cluster = MarkerCluster()

# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)



m.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)
    

# Iterate over each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m.add_child(circle)
    
    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    m.add_child(marker)

# Display the map
m

In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
import folium

# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678
railway_lat = 28.57205
railway_lon = -80.58527

# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)


marker_cluster = MarkerCluster()

# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)



m.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)
    

# Iterate over each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m.add_child(circle)
    
    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    m.add_child(marker)

# Display the map
m

In [ ]:
import folium

# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678
highway_lat = 28.56339
highway_lon = -80.57077

# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)


marker_cluster = MarkerCluster()

# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)



m.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)
    

# Iterate over each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m.add_child(circle)
    
    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    m.add_child(marker)

# Display the map
m

In [ ]:
import folium

# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678
city_lat = 28.40167
city_lon = -80.60394

# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)


marker_cluster = MarkerCluster()

# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)



m.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)
    

# Iterate over each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m.add_child(circle)
    
    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    m.add_child(marker)

# Display the map
m